In [1]:
import rasterio
import xarray as xr
import sys
import datacube

import numpy as np
from multiprocessing import Pool
import dask.array as da
import os

import dask
from dask.distributed import Client

sys.path.append('../lib')
import tsmask_func as tsf
import testpair as cym
import dea_datahandling as ddh



import tensorflow as tf
import tensorflow.keras.models as models
from datacube.utils.cog import write_cog


In [2]:
def load_bgids(indices_list, dirc, indstr, loc_str):
    
    
    
    n_ids = len(indices_list)
    
    print("Start loading long term mean of indices")

    bgids =[]
    for i, indname in enumerate(indices_list):
        fname = dirc + '/'+loc_str+'_'+indname+'_'+indstr+'.tif'
        dbs=xr.open_rasterio(fname)
        if i == 0:
            irow = dbs['y'].size
            icol = dbs['x'].size
            #bgids = np.zeros((n_ids, irow*icol), dtype = np.float32)
        
        #bgids[i] = dbs.data[0].flatten()
        bgids.append(dbs.data[0])
        dbs.close()
    
    return bgids


In [3]:
def prep_features(sa, mndwi, msavi, whi, bg_indices, ipdata):
    n_ids=4
    for k in range(n_ids):
        ipdata[: , k*3] = bg_indices[k, :]
        
    ipdata[:, 1] = sa
    ipdata[:, 4] = mndwi
    ipdata[:, 7] = msavi
    ipdata[:, 10] = whi
    
    for k in range(n_ids):
        ipdata[: , k*3+2] = (ipdata[: , k*3+1] - ipdata[: , k*3]) / ipdata[: , k*3]
        
    
    return ipdata    


In [4]:
def create_ip_data(chblue, chgreen, chred, chnir, chswir1, chswir, bgids, loc_str, outdirc):
    
    
    irow=s2_ds['y'].size
    icol=s2_ds['x'].size
    tn = s2_ds['time'].size

    timebandnames = tsf.get_timebandnames(s2_ds)
    
    pnum = irow * icol
    
    n_ids = bgids.shape[0]
    
    ipdata=np.zeros((pnum, n_ids*3), dtype=np.float32)

    for i in np.arange(tn):

        blue = s2_ds["blue"][i].values
        #print("Finish loading the blue band")  
    
        green = s2_ds["green"][i].values
        #print("Finish loading the green band")

        red = s2_ds["red"][i].values
        #print("Finish loading the red band")

        nir = s2_ds["nir"][i].values
        #print("Finish loading the nir band")

        swir1 = s2_ds["swir1"][i].values
        #print("Finish loading the swir1 band")

        swir2 = s2_ds["swir2"][i].values
        #print("Finish loading the swir2 band")
    
   
  
    
    
        #convert cal_indices in cython
        sa, mndwi, msavi, whi, mask = cym.cal_indices(blue.flatten(), green.flatten(), red.flatten(), nir.flatten(), swir1.flatten(), swir2.flatten(), pnum)

        print(timebandnames[i])
        ipdata = prep_features(sa, mndwi, msavi, whi, bgids, ipdata)
        #print(ipdata[:, 11])
        #print(i, mask.shape, mask.sum())
        vdipdata = ipdata[mask==1]
        #print(vdipdata.shape, vdipdata[:,11])
        vdipdata = vdipdata.flatten()


        datafname = outdirc + '/' + loc_str + '_'+timebandnames[i]+'_ipdata'
        np.save(datafname, vdipdata)
        maskfname = outdirc + '/' + loc_str + '_'+timebandnames[i]+'_ipmask'
        np.save(maskfname, mask)
        
    tsbandfname = outdirc + '/' + loc_str + '_timebandnames'
    np.save(tsbandfname, timebandnames)


In [5]:
def dumb_getipdata(blue, green, red, nir, swir1, swir2, s6m, mndwi, msavi, whi):
    irow = blue.shape[0]
    icol = blue.shape[1]
    ipdata = np.zeros((irow, icol, 12), dtype = np.float32 )
    ipdata[:, :, 0] = msavi
    
    return ipdata

In [6]:
def cal_ip_data(blue, green, red, nir, swir1, swir2, s6m, mndwi, msavi, whi):
    
    #tsmask = cym.tsmask(blue, green, red, nir, swir1, swir2)
    ipdata = cym.getipdata(blue, green, red, nir, swir1, swir2, s6m, mndwi, msavi, whi)
    #ipdata = dumb_getipdata(blue, green, red, nir, swir1, swir2, s6m, mndwi, msavi, whi)
    
    return ipdata

def tf_data(blue, green, red, nir, swir1, swir2, s6m, mndwi, msavi, whi):
    return xr.apply_ufunc(
        cal_ip_data, blue, green, red, nir, swir1, swir2, s6m, mndwi, msavi, whi,
        dask='parallelized',
        #input_core_dims=[["time"], ["time"],["time"], ["time"],["time"], ["time"]],
        #input_core_dims=[["time"], ["time"]],
        #vectorize = True,
        #output_core_dims=[],
        #output_core_dims= [[] for _ in range(4)], 
        output_core_dims= [['ipdata']], 
        #output_sizes = {'ipdata' : 13},
        #dask_gufunc_kwargs = {"allow_rechunk": True},
        dask_gufunc_kwargs = {'output_sizes' : {'ipdata' : 13}},
        output_dtypes = [np.float32]
        #output_dtypes = [np.uint8, np.uint8, np.uint8, np.uint8]
    )



In [7]:
def std_by_paramters(data, rs, msarr):
    ntr=data.shape[1]
    for i in np.arange(ntr):
        clm=data[:, i]
        mu=msarr[i]
        std=msarr[i+ntr]
        clm=(clm-mu)/(rs*std)
        data[:,i]=clm
        
    return data


In [8]:
dirc ='/home/jovyan/nmask_testdata/cbr_dask_run/indices'

loc_str ='canberra'

ncpu = 2


y1, y2 = -32.53284301899998, -33.52310232399998
x1, x2 = 121.934694247, 123.105109264

crs = 'EPSG:4326'
out_crs = 'UTM'

start_of_epoch = '2018-01-01'
end_of_epoch ='2020-12-31'

if out_crs == 'UTM':

    out_crs = tsf.utm_code(x1, x2)

#create_local_dask_cluster(spare_mem='4GB')

outdirc = '/home/jovyan/nmask_testdata/cbr_dask_run/maskfiles'
modeldirc ='/home/jovyan/nmask_dask/models'
modelname ='combine_trdata_tsmask_model'

parafilename=modeldirc+'/'+modelname+'_standarise_parameters.npy'
norm_paras = np.load(parafilename)
modelfilename=modeldirc+'/'+modelname
model = models.load_model(modelfilename)


client = Client(n_workers = ncpu, threads_per_worker=1, processes = True)
client



Client Scheduler: tcp://127.0.0.1:41657 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 16.11 GB


In [9]:
indices_list=['s6m', 'mndwi', 'msavi', 'whi']
indstr = start_of_epoch+"_"+end_of_epoch


bgids = load_bgids(indices_list, dirc, indstr, loc_str)


indfile=dirc+'/'+loc_str+'_msavi_'+indstr+'.tif'
dbs=xr.open_rasterio(indfile)



Start loading long term mean of indices


In [10]:
bgids[2].shape

(5515, 5497)

In [11]:
chy, chx = 500, 500

dc = datacube.Datacube(app='load_clearsentinel')
tg_ds=tsf.load_s2_nbart_dask(dc, y1, y2, x1, x2, start_of_epoch, end_of_epoch, {   
        "time": 1, "y": chy, "x" : chx }, crs, out_crs )


/env/lib/python3.6/site-packages/datacube/drivers/postgres/_connections.py:87: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  username=username, password=password,


Finding datasets
    s2a_ard_granule
    s2b_ard_granule
Returning 424 time steps as a dask array


In [12]:
for i, indname in enumerate(indices_list):
    tg_ds[indname]=tg_ds['blue'][0]
    tg_ds[indname].data = bgids[i]
    tg_ds[indname]=tg_ds[indname].chunk({'y':chy, 'x':chx})

In [ ]:


 # number of rows
irow=tg_ds['y'].size
# number of columns
icol=tg_ds['x'].size
# number of time steps
tn = tg_ds['time'].size

tbnamelist = tsf.get_timebandnames(tg_ds)

for i in range(tn):
   
    nmask = np.zeros(irow*icol, dtype=np.uint8)
    blue = tg_ds.blue[i, :, :].persist()
    green = tg_ds.green[i, :, :].persist()
    red = tg_ds.red[i, :, :].persist()
    nir = tg_ds.nir[i, :, :].persist()
    swir1 = tg_ds.swir1[i, :, :].persist()
    swir2 = tg_ds.swir2[i, :, :].persist()
    s6m = tg_ds.s6m.persist()
    mndwi = tg_ds.mndwi.persist()
    msavi = tg_ds.msavi.persist()
    whi = tg_ds.whi.persist()
    
    ipdata = tf_data(blue, green, red, nir, swir1, swir2, s6m, mndwi, msavi, whi).compute()
    
    #print(ipdata[:, :, 6].data, ipdata.shape)
    
    #print(ipdata[:, :, 6].data - bgids[2])
    
    
    
    ipmask = ipdata[:, :, 12].data
    tfdata = ipdata[:, :, 0:12].data
    
    ipmask = ipmask.flatten().astype(np.int)
    tfdata = tfdata.reshape(irow*icol, 12)
    
    tfdata = tfdata[ipmask == 1]
    tfdata = std_by_paramters(tfdata, 2, norm_paras)
    
    tbname = tbnamelist[i]
    print("Begin classifying scene ", tbname)
    
    mixtures=model.predict(tfdata)
    
    vdmask = np. argmax(mixtures, axis = 1) + 1
    
    nmask[ipmask==1] = vdmask
    nmask = nmask.reshape(irow, icol)
    
    nmask =  cym.spatial_filter_v2(nmask)
    
    dbs.data[0] = nmask
    
    outfname = outdirc+'/'+loc_str+'_'+tbname+'_nmask-cog.tif'
    write_cog(geo_im = dbs, fname = outfname, overwrite = True)
  
    
    print("Finish writing mask file ", outfname)
    
    

Begin classifying scene  2018-01-01


/env/lib/python3.6/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


Finish writing mask file  /home/jovyan/nmask_testdata/cbr_dask_run/maskfiles/canberra_2018-01-01_nmask-cog.tif
Begin classifying scene  2018-01-06
Finish writing mask file  /home/jovyan/nmask_testdata/cbr_dask_run/maskfiles/canberra_2018-01-06_nmask-cog.tif
Begin classifying scene  2018-01-08
Finish writing mask file  /home/jovyan/nmask_testdata/cbr_dask_run/maskfiles/canberra_2018-01-08_nmask-cog.tif


distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


Begin classifying scene  2018-01-11
Finish writing mask file  /home/jovyan/nmask_testdata/cbr_dask_run/maskfiles/canberra_2018-01-11_nmask-cog.tif
Begin classifying scene  2018-01-13


...

Begin classifying scene  2018-04-21
Finish writing mask file  /home/jovyan/nmask_testdata/cbr_dask_run/maskfiles/canberra_2018-04-21_nmask-cog.tif
Begin classifying scene  2018-04-23


distributed.nanny - WARNING - Restarting worker


Finish writing mask file  /home/jovyan/nmask_testdata/cbr_dask_run/maskfiles/canberra_2018-04-23_nmask-cog.tif
Begin classifying scene  2018-04-26
Finish writing mask file  /home/jovyan/nmask_testdata/cbr_dask_run/maskfiles/canberra_2018-04-26_nmask-cog.tif
Begin classifying scene  2018-04-28


distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


Finish writing mask file  /home/jovyan/nmask_testdata/cbr_dask_run/maskfiles/canberra_2018-04-28_nmask-cog.tif
